In [1]:
import pandas as pd
import datetime
import numpy as np
import os.path
import wget
from zipfile import ZipFile
import shutil
import re
from multiprocessing import Pool
import multiprocessing as mp

In [2]:
## Variaveis globais

#qual base de dados quero usar, "con" para consolidado e "ind" para individual
tipo = "con"

#Pega data corrente e ano corrente
date = datetime.date.today()
year = date.year

# ano inicial pra coleta dos dados
year_0 = 2010 
year_0_tri = year-2 # ano inicial pra coletoa dos dados trimestrais

In [3]:
# baixa todos os dados se não estiverem presentes, retorna uma lista de nomes de arquivos baixados
def baixa_dados(year_0=year_0, year=year, year_0_tri=year_0_tri):
    # URL's onde estão os arquivos dfp e itr na CVM
    url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'
    url_base_fre = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/'
    url_base_tri = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
    url_base_fca = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FCA/DADOS/"

    # Cria uma lista com os nomes dos arquivos de dados anuais (dfp) da CVM
    arquivos_zip = []
    for ano in range(year_0, year + 1):
        arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')

    # baixa os arquivos dfp
    arquivos_zip_dfp_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_dfp_baixados.append(arq)
        else:
            try:
                wget.download(url_base + arq, out=f"./Data/{arq}")
                arquivos_zip_dfp_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # Cria lista de arquivos ITR (trimestrais) pra baixar
    arquivos_zip = []
    for ano in range(year_0_tri, year + 1):
        arquivos_zip.append(f'itr_cia_aberta_{ano}.zip')

    # baixa os arquivos ITR (dados trimestrais) se não estiverem presentes
    arquivos_zip_itr_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_itr_baixados.append(arq)
        else:
            try:
                wget.download(url_base_tri + arq, out=f"./Data/{arq}")
                arquivos_zip_itr_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # baixa arquivo FCA, que contém os tickers

    # Cria lista de arquivos FCA  pra baixar. Este aqui só precisa baixar o do último ano disponível
    # Se não tiver do ano atual baixa o do ano passado
    arquivos_zip = []
    for ano in [year, year - 1]:
        arquivos_zip.append(f'fca_cia_aberta_{ano}.zip')

    arquivos_zip_fca_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_fca_baixados.append(arq)
            break
        else:
            try:
                wget.download(url_base_fca + arq, out=f"./Data/{arq}")
                arquivos_zip_fca_baixados.append(arq)
                break # se conseguiu baixar 1 já sai do loop
            except:
                print(f"falha ao baixar {arq}")
    
    # baixa arquivo FRE, que contém o número total de ações
    
    arquivos_zip = []
    for ano in range(year-3, year+1):
        arquivos_zip.append(f'fre_cia_aberta_{ano}.zip')

    arquivos_zip_fre_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_fre_baixados.append(arq)
        else:
            try:
                wget.download(url_base_fre + arq, out=f"./Data/{arq}")
                arquivos_zip_fre_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                
    return (arquivos_zip_dfp_baixados, arquivos_zip_itr_baixados, arquivos_zip_fca_baixados, arquivos_zip_fre_baixados)

In [4]:
def processa_dados(year_0=year_0, year=year, year_0_tri=year_0_tri):
    print("Processando dados")
    
    # DFP - Dados anuais
    # junta todos os anos de dados anuais em um arquivo só de cada tipo
    nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
             'DFC_MD_con', 'DFC_MI_con', 'DFC_MD_ind', 'DFC_MI_ind']
    for nome in nomes:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0, year):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/dfp_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em {nome}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/dfp_cia_aberta_{nome}_{year_0}-{year}.csv', index=False, encoding="latin1")
        print(f'DFP {nome} processado')
    
    # ITR - Dados trimestrais
    # junta todos os anos de dados trimestrais em um arquivo só de cada tipo
    nomes_tri = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
                 "DFC_MD_con", "DFC_MI_con", "DFC_MD_ind", "DFC_MI_ind"]
    for nome in nomes_tri:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0_tri, year + 1):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/itr_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em itr_cia_aberta_{nome}_{ano}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/itr_cia_aberta_{nome}_{year_0_tri}-{year}.csv', index=False, encoding="latin1")
        print(f'ITR {nome} processado')
    
    # FRE - Contém q qtd de ações das empresas
    
    # cria uma lista com todos os arquivos fre
    arq_fre = [item for item in os.listdir("./Data/temp") if item.startswith("fre_cia_aberta_distribuicao_capital_2")]
    fre = pd.DataFrame()

    # concatena tudo
    for arq in arq_fre[::-1]:
        fre = pd.concat([fre, pd.read_csv(f"./Data/temp/{arq}", sep=";")], axis=0)

    # joga fora as linhas com 0
    fre = fre[fre["Quantidade_Total_Acoes_Circulacao"] != 0]

    # ordena pelo mais recente, retira as duplicatas e pega só as colunas que interessam
    fre = fre.sort_values(by=["Data_Referencia", "Versao"], ascending=False).drop_duplicates(["CNPJ_Companhia"], keep="first", ignore_index=True)[["CNPJ_Companhia", "Quantidade_Total_Acoes_Circulacao"]]

    # salva em um csv
    fre.to_csv("./Data/processados/qtd_acoes.csv", encoding="latin1", index=False)
    fre.set_index("CNPJ_Companhia", inplace=True)
    print("FRE processado")
    
    # arquivo FCA - Contém tickers das empresas
    
    # pega o nome do arquivo fca que contem os tickers
    arq_fca = [item for item in os.listdir("./Data/temp") if item.startswith("fca_cia_aberta_valor_mobiliario")]
    arq_fca = arq_fca[0]

    # copia o arquivo fca pra pasta processados
    shutil.copyfile(f"./Data/temp/{arq_fca}", f"./Data/processados/fca.csv")
    print("FCA processado")
        
    print("Arquivos processados")

In [5]:
# só extrai o arquivo dado como parâmetro para a pasta ./Data/temp
def extrai_arquivo(arq):
    try:
        ZipFile(f"./Data/{arq}", 'r').extractall('./Data/temp')
        print(f"{arq} extraido")
    except:
        print(f"erro ao extrair {arq}")

# extrai uma lista de arquivos com paralelismo. não roda no jupyter

def extrai_paralelo(arquivos_para_extrair):

    #  descobre quantas threads tem o CPU atual e cria uma pool para extrair os dados com paralelismo
    procs = mp.cpu_count()
    pool = Pool(procs)

    # cria uma lista de pools para depois dar get
    lista_pool = []
    for arq in arquivos_para_extrair:
        lista_pool.append(pool.apply_async(extrai_arquivo, args=(arq,)))
    print("Extraindo")
    output = [item.get() for item in lista_pool]
    return(output)

In [6]:
# faz o usuario escolher uma empresa
def escolhe_empresa(lista_empresas):
    while True:
        busca = input("Entre com o ticker da empresa sem o número: ")
        empresa = lista_empresas[lista_empresas['TICKER'].str.contains(busca, case=False)]
        print(empresa[["DENOM_CIA", "CD_CVM", "TICKER"]])
        ok = input("A empresa desejada está na primeira linha? (y/n) ")
        if ok == "y":
            return(empresa)

In [7]:
# retorna um df com CNPJ razao social e ticker de todas as empresas listadas em bolsa
def cria_lista_empresas(year_0=year_0, year=year, year_0_tri=year_0_tri):

    #cria lista de empresas. Concatena consolidado e individual
    lista_empresas = pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_con_{year_0}-{year}.csv", header=0, sep=",", encoding="latin1")[["DENOM_CIA", 'CD_CVM', "CNPJ_CIA"]].drop_duplicates()
    lista_empresas["tipo"] = "con"
    ind = pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_ind_{year_0}-{year}.csv", header=0, sep=",", encoding="latin1")[["DENOM_CIA", 'CD_CVM', "CNPJ_CIA"]].drop_duplicates()
    ind["tipo"] = "ind"
    lista_empresas = pd.concat([lista_empresas, ind], ignore_index=True).drop_duplicates(keep="first")
    lista_empresas.set_index("CNPJ_CIA", inplace=True)

    # coloca tickers do arquivo fca
    lista_empresas = lista_empresas.join(pd.read_csv(f"./Data/processados/fca.csv", header=0, sep=";", encoding="latin1").set_index("CNPJ_Companhia")["Codigo_Negociacao"].dropna())

    # pega tickers da B3 tbm pq na CVM é incompleto.
    # extraí de https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm
    # no futuro seria bom automatizar isso com um webscraper
    b3 = pd.read_csv("./Data/empresas_listadas_b3.csv", encoding="latin1", sep=";")[["Razão Social","Código"]]

    # Cria a coluna CNPJ com os dados do index
    lista_empresas = lista_empresas.rename_axis('CNPJ').reset_index()

    # Junta os dados da b3 no arquivo original
    lista_empresas = lista_empresas.merge(b3, left_on="DENOM_CIA", right_on="Razão Social", how="left").drop(columns=["Razão Social"])
    
    # Dá replace em todas as linhas de "Codigo_Negociacao" que não obedecem o padrão "ABCD1" ou "ABCD11"
    lista_empresas["Codigo_Negociacao"].replace(to_replace= r"^(?!\D{4}\d{1,2}$).*", value= np.nan, regex=True, inplace=True)

    # Pega só as 4 primeiras letras da coluna "Codigo_Negociacao"
    lista_empresas["TICKER"] = lista_empresas["Codigo_Negociacao"].str[:4]
    lista_empresas.drop(columns=["Codigo_Negociacao"], inplace=True)
    lista_empresas.drop_duplicates(inplace=True)

    # Dá replace nos NaN da CVM com o ticker do site da B3
    lista_empresas["TICKER"].fillna(lista_empresas["Código"], inplace=True)

    # Joga fora a coluna com tickers da B3
    lista_empresas.drop(columns=["Código"], inplace=True)
    
    # joga fora o que não tiver ticker
    lista_empresas.dropna(inplace=True)
    
    #deixa o index como o CNPJ
    lista_empresas.set_index("CNPJ")
    
    # tira linhas duplicadas (tem que ter CNPJ, CD_CVM e TICKER iguais)
    lista_empresas.drop_duplicates(["CNPJ", "CD_CVM", "TICKER"], inplace=True)
    
    #salva em um csv
    lista_empresas.to_csv("./Data/processados/lista empresas.csv", encoding="latin1", index=False)
    
    return(lista_empresas)

## Main

In [8]:
# Limpeza

# Deleta o conteúdo da pasta processados se existir
lista_arquivos = os.listdir("./Data/processados/")
lista_arquivos.remove(".gitignore")

for item in lista_arquivos:
    try:
        os.remove(f"./Data/processados/{item}")
    except:
        pass

#deleta pasta temporaria se existir
try:
    shutil.rmtree(f'./Data/temp')
except:
    pass

In [9]:
# baixa os dados e coloca os nomes dos arquivos baixados em cada variavel
arquivos_dfp, arquivos_itr, arquivos_fca, arquivos_fre = baixa_dados(year_0, year, year_0_tri)
arquivos_dfp, arquivos_itr, arquivos_fca, arquivos_fre

dfp_cia_aberta_2010.zip já esta aqui
dfp_cia_aberta_2011.zip já esta aqui
dfp_cia_aberta_2012.zip já esta aqui
dfp_cia_aberta_2013.zip já esta aqui
dfp_cia_aberta_2014.zip já esta aqui
dfp_cia_aberta_2015.zip já esta aqui
dfp_cia_aberta_2016.zip já esta aqui
dfp_cia_aberta_2017.zip já esta aqui
dfp_cia_aberta_2018.zip já esta aqui
dfp_cia_aberta_2019.zip já esta aqui
dfp_cia_aberta_2020.zip já esta aqui
dfp_cia_aberta_2021.zip já esta aqui
dfp_cia_aberta_2022.zip já esta aqui
itr_cia_aberta_2020.zip já esta aqui
itr_cia_aberta_2021.zip já esta aqui
itr_cia_aberta_2022.zip já esta aqui
fca_cia_aberta_2022.zip já esta aqui
fre_cia_aberta_2019.zip já esta aqui
fre_cia_aberta_2020.zip já esta aqui
fre_cia_aberta_2021.zip já esta aqui
fre_cia_aberta_2022.zip já esta aqui


(['dfp_cia_aberta_2010.zip',
  'dfp_cia_aberta_2011.zip',
  'dfp_cia_aberta_2012.zip',
  'dfp_cia_aberta_2013.zip',
  'dfp_cia_aberta_2014.zip',
  'dfp_cia_aberta_2015.zip',
  'dfp_cia_aberta_2016.zip',
  'dfp_cia_aberta_2017.zip',
  'dfp_cia_aberta_2018.zip',
  'dfp_cia_aberta_2019.zip',
  'dfp_cia_aberta_2020.zip',
  'dfp_cia_aberta_2021.zip',
  'dfp_cia_aberta_2022.zip'],
 ['itr_cia_aberta_2020.zip',
  'itr_cia_aberta_2021.zip',
  'itr_cia_aberta_2022.zip'],
 ['fca_cia_aberta_2022.zip'],
 ['fre_cia_aberta_2019.zip',
  'fre_cia_aberta_2020.zip',
  'fre_cia_aberta_2021.zip',
  'fre_cia_aberta_2022.zip'])

In [10]:
# extrai arquivos
arquivos_para_extrair = arquivos_dfp + arquivos_itr + arquivos_fca + arquivos_fre

# extrai sequencial
for arq in arquivos_para_extrair:
    extrai_arquivo(arq)

# extrai paralelo, não funciona no jupyter
#extrai_paralelo(arquivos_para_extrair)

dfp_cia_aberta_2010.zip extraido
dfp_cia_aberta_2011.zip extraido
dfp_cia_aberta_2012.zip extraido
dfp_cia_aberta_2013.zip extraido
dfp_cia_aberta_2014.zip extraido
dfp_cia_aberta_2015.zip extraido
dfp_cia_aberta_2016.zip extraido
dfp_cia_aberta_2017.zip extraido
dfp_cia_aberta_2018.zip extraido
dfp_cia_aberta_2019.zip extraido
dfp_cia_aberta_2020.zip extraido
dfp_cia_aberta_2021.zip extraido
dfp_cia_aberta_2022.zip extraido
itr_cia_aberta_2020.zip extraido
itr_cia_aberta_2021.zip extraido
itr_cia_aberta_2022.zip extraido
fca_cia_aberta_2022.zip extraido
fre_cia_aberta_2019.zip extraido
fre_cia_aberta_2020.zip extraido
fre_cia_aberta_2021.zip extraido
fre_cia_aberta_2022.zip extraido


In [13]:
# concatena os arquivos baixados do ano inicial até o final
processa_dados(year_0, year, year_0_tri)

#deleta pasta temporaria se existir
try:
    shutil.rmtree(f'./Data/temp')
except:
    pass

Processando dados
DFP BPA_con processado
DFP BPA_ind processado
DFP BPP_con processado
DFP BPP_ind processado
DFP DRE_con processado
DFP DRE_ind processado
DFP DFC_MD_con processado
DFP DFC_MI_con processado
DFP DFC_MD_ind processado
DFP DFC_MI_ind processado
ITR BPA_con processado
ITR BPA_ind processado
ITR BPP_con processado
ITR BPP_ind processado
ITR DRE_con processado
ITR DRE_ind processado
ITR DFC_MD_con processado
ITR DFC_MI_con processado
ITR DFC_MD_ind processado
ITR DFC_MI_ind processado
FRE processado
FCA processado
Arquivos processados


## Fim do processamento dos dados

In [10]:
# cria um dataframe com todas as empresas listadas e ticker
lista_empresas = cria_lista_empresas(year_0, year, year_0_tri)

# muda o index pro CNPJ pra poder dar join
lista_empresas.set_index("CNPJ", inplace=True)

# coloca a quantidade de ações em circulação e tira as empresas que não tem. limpa bem os dados
fre = pd.read_csv("./Data/processados/qtd_acoes.csv").set_index("CNPJ_Companhia")
lista_empresas = lista_empresas.join(fre).dropna()
lista_empresas

,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao
00.000.000/0001-91,BCO BRASIL S.A.,1023,con,BBAS,1.420731e+09
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,con,BSLI,4.195216e+07
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,con,ELET,5.997421e+08
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,con,CEBR,1.430870e+07
00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,con,ARML,1.718094e+08
...,...,...,...,...,...
94.813.102/0001-70,TRÊS TENTOS AGROINDUSTRIAL S/A,25950,ind,TTEN,1.123436e+08
95.426.862/0001-97,EXCELSIOR ALIMENTOS S.A.,1570,ind,BAUH,1.508636e+06
96.418.264/0218-02,LOJAS QUERO QUERO S.A.,25038,con,LJQQ,1.789674e+08
97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,con,ODER,1.979520e+05


In [65]:
empresa = escolhe_empresa(lista_empresas)
empresa

Entre com o ticker da empresa sem o número: rrrp
                                       DENOM_CIA  CD_CVM TICKER
12.091.809/0001-55  3R PETROLEUM ÓLEO E GÁS S.A.   25291   RRRP
A empresa desejada está na primeira linha? (y/n) y


,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao
12.091.809/0001-55,3R PETROLEUM ÓLEO E GÁS S.A.,25291,con,RRRP,200453863.0


In [66]:
cnpj_selecionado = empresa.index[0]
tipo_empresa_selecionada = empresa["tipo"].iloc[0]
cnpj_selecionado, tipo_empresa_selecionada

('12.091.809/0001-55', 'con')

In [ ]:
# calcular indicadores e colocar cotação (adiar ao maximo pq demora mto tempo pra pegar)

In [11]:
# Cria a coluna CNPJ com os dados do index
#lista_empresas = lista_empresas.rename_axis('CNPJ').reset_index()
#lista_empresas

,CNPJ,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao
0,00.000.000/0001-91,BCO BRASIL S.A.,1023,con,BBAS,1.420731e+09
1,00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,con,BSLI,4.195216e+07
2,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,con,ELET,5.997421e+08
3,00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,con,CEBR,1.430870e+07
4,00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,con,ARML,1.718094e+08
...,...,...,...,...,...,...
407,94.813.102/0001-70,TRÊS TENTOS AGROINDUSTRIAL S/A,25950,ind,TTEN,1.123436e+08
408,95.426.862/0001-97,EXCELSIOR ALIMENTOS S.A.,1570,ind,BAUH,1.508636e+06
409,96.418.264/0218-02,LOJAS QUERO QUERO S.A.,25038,con,LJQQ,1.789674e+08
410,97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,con,ODER,1.979520e+05


In [11]:
# cria arquivo dre
dre = pd.concat([pd.read_csv(f'./Data/processados/dfp_cia_aberta_DRE_{tipo}_{year_0}-{year}.csv', encoding="latin1"),
                 pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")],
               ignore_index=True).drop_duplicates()
dre = dre[dre["ORDEM_EXERC"] == "ÚLTIMO"]
dre.set_index("CNPJ_CIA", inplace=True)
dre

,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
CNPJ_CIA,,,,,,,,,,,,,,
00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receitas da Intermediação Financeira,8.514321e+07,S
00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01.01,Receita de Juros,8.514321e+07,S
00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.02,Despesas da Intermediação Financeira,-5.077574e+07,S
00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.02.01,Despesa de Juros,-4.306119e+07,S
00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.02.02,Despesa com Provisão para Perdas em Empréstimos,-7.714554e+06,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97.837.181/0001-47,2022-03-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2022-01-01,2022-03-31,3.99,Lucro por Ação - (Reais / Ação),0.000000e+00,N
97.837.181/0001-47,2022-03-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2022-01-01,2022-03-31,3.99.01,Lucro Básico por Ação,0.000000e+00,N
97.837.181/0001-47,2022-03-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2022-01-01,2022-03-31,3.99.01.01,ON,3.011000e-01,N


In [99]:
#for tipo in ["con", "ind"]:
   # dre = pd.concat([pd.read_csv(f'./Data/processados/dfp_cia_aberta_DRE_{tipo}_{year_0}-{year}.csv', encoding="latin1"),
   #              pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")],
  #             ignore_index=True).drop_duplicates()
   # dre = dre[dre["ORDEM_EXERC"] == "ÚLTIMO"]
   # dre.set_index("CNPJ_CIA", inplace=True)
        #for cnpj in lista_empresas["CNPJ"]:
cnpj = cnpj_selecionado
lucros = dre[(dre.index == cnpj) & 
             ((dre.CD_CONTA == "3.09") | (dre.CD_CONTA == "3.11") ) & 
             ((dre.DS_CONTA == 'Lucro/Prejuízo do Período') | (dre.DS_CONTA == 'Lucro ou Prejuízo Líquido Consolidado do Período') | ((dre.DS_CONTA == 'Lucro/Prejuízo Consolidado do Período')))
            ][[ "DT_INI_EXERC", "DT_FIM_EXERC","VL_CONTA"]].sort_values(by=["DT_INI_EXERC", "DT_FIM_EXERC"])
lucros_ajeitado = pd.DataFrame({"CNPJ" : cnpj}, index=[0])
lucros_ajeitado.set_index("CNPJ", inplace=True)
# for que passa em todos os tri
for ano in range(year_0, year+1): # pra todos os anos
    # essas listas no for são pra ele pegar a data base correta pra cada trimestre. 
    # ex: tri_ini = 04 e tri_fim = 06 resulta em dados do 2T, que sempre começam no mes 4 e acabam no ultimo dia do mês 6
    # não existe 4T nos dados e precisa ser calculado. no último valor uso do mês 01 até 12, que pega os dados anuais. 
    # Deixo sem nada no último valor de nome_tri pra ficar só o ano no nome da coluna anual
    for tri_ini, tri_fim, nome_tri in zip(["01", "04", "07", "10", "01"], ["03", "06", "09", "12", "12"], ["1T", "2T", "3T", "4T", ""]):
        #print(tri_ini, tri_fim, nome_tri, ano)
        lucros_ajeitado = lucros_ajeitado.join((lucros[(lucros.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-3")) & 
                                  (lucros.DT_INI_EXERC == (f"{ano}-{tri_ini}-01"))][["VL_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"})))
    # calcula resultado do 4T
    lucros_ajeitado[f"4T{ano}"] = lucros_ajeitado[f"{ano}"] - lucros_ajeitado[f"3T{ano}"] - lucros_ajeitado[f"2T{ano}"] - lucros_ajeitado[f"1T{ano}"]
# o for resulta em colunas de NaN pros anos que não tem dados, removo a coluna inteira quando tem um NaN nela. 
# Isso pode dar problema se estiver faltando apenas um dado para algum ano/tri, pois vai remover o ano/tri inteiro. 
# Não sei se ocorre isso na prática, vamos ver.
lucros_ajeitado.dropna(axis=1, inplace=True)
ultimos_4_tri = [item for item in list(lucros_ajeitado.columns) if (bool(re.search(r"^(\dT)", item)) and ((item.endswith(str(year))) or (item.endswith(str(year-1)))))][-4:]
print(ultimos_4_tri)
if len(ultimos_4_tri) == 4:
    lucros_ajeitado["TTM"] = lucros_ajeitado[ultimos_4_tri].sum(axis=1)
lucros_ajeitado

['2T2021', '3T2021', '4T2021', '1T2022']


,2019,1T2020,2T2020,3T2020,4T2020,2020,1T2021,2T2021,3T2021,4T2021,2021,1T2022,TTM
CNPJ,,,,,,,,,,,,,
12.091.809/0001-55,-31966.0,-27952.0,-42319.0,-12749.0,-193510.0,-276530.0,-43974.0,54286.0,-14101.0,19780.0,15991.0,-335175.0,-275210.0


In [ ]:
# Solução com todas as linhas ao mesmo tempo
# pensando bem, dados trimestrais só servem pra calcular o TTM, depois disso dá pra jogar fora

In [40]:
lista_empresas_con = lista_empresas[lista_empresas["tipo"] == "con"]
lista_empresas_ind = lista_empresas[lista_empresas["tipo"] == "ind"]

In [41]:
# coloca as colunas de lucro com o nome da coluna correspondendo ao trimestre/ano
for ano in range(year_0, year+1):
    for tri_ini, tri_fim, nome_tri in zip(["01", "04", "07", "10", "01"], ["03", "06", "09", "12", "12"], ["1T", "2T", "3T", "4T", ""]):
        lista_empresas_con = lista_empresas_con.join(dre[(dre.DT_INI_EXERC == (f"{ano}-{tri_ini}-01")) & 
                                                         (dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-3")) & 
                                                         (((dre.DS_CONTA == 'Lucro/Prejuízo Consolidado do Período')) | (dre.DS_CONTA == 'Lucro ou Prejuízo Líquido Consolidado do Período'))][
            ["VL_CONTA", "DS_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"}).drop("DS_CONTA", axis=1))
        
    lista_empresas_con[f"4T{ano}"] = lista_empresas_con[f"{ano}"] - lista_empresas_con[f"3T{ano}"] - lista_empresas_con[f"2T{ano}"] - lista_empresas_con[f"1T{ano}"]
lista_empresas_con.dropna(axis=1, inplace=True, how="all")
lista_empresas_con

,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao,2010,1T2011,2011,2012,2013,...,2T2020,3T2020,4T2020,2020,1T2021,2T2021,3T2021,4T2021,2021,1T2022
00.000.000/0001-91,BCO BRASIL S.A.,1023,con,BBAS,1.420731e+09,11330345.0,NaN,12736912.0,11438200.0,11288834.0,...,NaN,NaN,NaN,13292883.0,NaN,NaN,NaN,NaN,19722871.0,4974664.0
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,con,BSLI,4.195216e+07,262184.0,NaN,128936.0,195644.0,211225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,con,ELET,5.997421e+08,2552985.0,NaN,3762019.0,-6925652.0,-6291375.0,...,4597353.0,95764.0,1387360.0,6387313.0,1609139.0,2529924.0,964561.0,610009.0,5713633.0,2716375.0
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,con,CEBR,1.430870e+07,19617.0,NaN,46132.0,74679.0,-75163.0,...,-45270.0,106012.0,406670.0,465378.0,1151368.0,27544.0,39743.0,75752.0,1294407.0,53590.0
00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,con,ARML,1.718094e+08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13676.0,NaN,58064.0,27965.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92.781.335/0001-02,TAURUS ARMAS S.A.,6173,con,TASA,7.003206e+07,70310.0,NaN,37337.0,-117210.0,-80310.0,...,39035.0,102235.0,279457.0,263603.0,68096.0,193631.0,166411.0,206922.0,635060.0,194993.0
92.791.243/0001-03,IRANI PAPEL E EMBALAGEM S.A.,2429,con,RANI,8.961692e+07,34361.0,NaN,9359.0,26383.0,67410.0,...,15298.0,25558.0,33917.0,92761.0,56701.0,67691.0,97595.0,63326.0,285313.0,112148.0
96.418.264/0218-02,LOJAS QUERO QUERO S.A.,25038,con,LJQQ,1.789674e+08,NaN,NaN,NaN,NaN,NaN,...,4447.0,30105.0,34852.0,67866.0,11612.0,15986.0,15551.0,25078.0,68227.0,-10264.0
97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,con,ODER,1.979520e+05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,37140.0,NaN,96095.0,39527.0


In [42]:
# cria coluna TTM
for cnpj in lista_empresas_con.index:
    lista_empresas_con
    ultimos_4_tri = [item for item in list(lista_empresas_con.loc[cnpj].dropna().index) if (bool(re.search(r"^(\dT)", item)) and ((item.endswith(str(year))) or (item.endswith(str(year-1)))))][-4:]
    if len(ultimos_4_tri) == 4:
        lista_empresas_con.loc[cnpj, "TTM"] = lista_empresas_con.loc[cnpj, ultimos_4_tri].sum()
lista_empresas_con.to_excel("teste.xlsx")

In [ ]:
# cria arquivo dre ind
tipo = "ind"
dre = pd.concat([pd.read_csv(f'./Data/processados/dfp_cia_aberta_DRE_{tipo}_{year_0}-{year}.csv', encoding="latin1"),
                 pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")],
               ignore_index=True).drop_duplicates()
dre = dre[dre["ORDEM_EXERC"] == "ÚLTIMO"]
dre.set_index("CNPJ_CIA", inplace=True)
dre

In [154]:
# tem que fazer o dre do individual antes
for ano in range(year_0, year+1):
    for tri_ini, tri_fim, nome_tri in zip(["01", "04", "07", "10", "01"], ["03", "06", "09", "12", "12"], ["1T", "2T", "3T", "4T", ""]):
        lista_empresas_ind = lista_empresas_ind.join(dre[(dre.DT_INI_EXERC == (f"{ano}-{tri_ini}-01")) & 
                                                         (dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-3")) & 
                                                         ((dre.DS_CONTA == 'Lucro/Prejuízo do Período') | ((dre.DS_CONTA == 'Lucro ou Prejuízo Líquido do Período')))][
            ["VL_CONTA", "DS_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"}).drop("DS_CONTA", axis=1))
lista_empresas_ind.dropna(axis=1, inplace=True, how="all")
lista_empresas_ind

,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao,1T2010,2T2010,3T2010,4T2010,2010,...,1T2021,2T2021,3T2021,4T2021,2021,1T2022,2T2022,3T2022,4T2022,2022
00.938.574/0001-05,CONC RIO-TERESOPOLIS S.A.,19208,ind,CRTE,86466.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01.104.937/0001-70,ELETROBRÁS PARTICIPAÇÕES S.A. - ELETROPAR,15784,ind,LIPR,1915985.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01.417.222/0001-77,MRS LOGISTICA S.A.,17949,ind,MRSA,6261278.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01.543.032/0001-04,CELG DISTRIBUICAO S.A. - CELG D,25577,ind,CGOS,102022.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01.548.981/0001-79,INVESTIMENTOS BEMGE S.A.,6041,ind,FIGE,42570.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91.669.747/0001-92,FINANSINOS S.A.- CREDITO FINANC E INVEST,6076,ind,FNCN,300101.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92.749.225/0001-63,HERCULES S.A. FABRICA DE TALHERES,6629,ind,HETA,243750.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93.828.986/0001-73,CEMEPE INVESTIMENTOS S.A.,13854,ind,MAPT,35693.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94.813.102/0001-70,TRÊS TENTOS AGROINDUSTRIAL S/A,25950,ind,TTEN,112343586.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
lista_empresas_ind.to_csv("teste.csv")

In [122]:
ano = 2020
tri_fim = "03"
tri_ini = "01"
dre[(dre.DT_INI_EXERC == (f"{ano}-{tri_ini}-01")) & 
    (dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-3")) & 
    ((dre.DS_CONTA == 'Lucro/Prejuízo do Período') | ((dre.DS_CONTA == 'Lucro/Prejuízo Consolidado do Período')) | (dre.DS_CONTA == 'Lucro ou Prejuízo Líquido Consolidado do Período') | ((dre.DS_CONTA == 'Lucro ou Prejuízo Líquido do Período')))][
    ["VL_CONTA", "DS_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"}).drop("DS_CONTA", axis=1)

,1T2020
CNPJ_CIA,
00.001.180/0001-26,306836.0
00.070.698/0001-11,-2034.0
00.359.742/0001-08,3714.0
00.622.416/0001-41,34079.0
00.776.574/0001-56,-107963.0
...,...
92.781.335/0001-02,-157124.0
92.791.243/0001-03,17988.0
96.298.013/0001-68,11049.0


In [117]:
# ao inves de fazer 1 por um poderia fazer pra varias linhas ao mesmo tempo
# ex: pra todas as linhas que tem tri_ini 01 e tri_fim 03 e ano 2020 crio uma coluna 1T2020
# precisa entender se o consolidado e individual vai afetar ou não
#for tri_ini, tri_fim, nome_tri in zip(["01", "04", "07", "10", "01"], ["03", "06", "09", "12", "12"], ["1T", "2T", "3T", "4T", ""]):
#    lista_empresas[f"{nome_tri}{ano}"] =

lista_empresas.join(dre[(dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-3")) & (lucros.DT_INI_EXERC == (f"{ano}-{tri_ini}-01"))][["VL_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"}))

In [133]:
ano = 2020
tri_fim = "03"
tri_ini = "01"
dre[(dre.DT_INI_EXERC == (f"{ano}-{tri_ini}-01")) & 
    (dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-3")) & 
    ((dre.DS_CONTA == 'Lucro/Prejuízo do Período') | ((dre.DS_CONTA == 'Lucro/Prejuízo Consolidado do Período')) | (dre.DS_CONTA == 'Lucro ou Prejuízo Líquido Consolidado do Período') | ((dre.DS_CONTA == 'Lucro ou Prejuízo Líquido do Período')))][
    ["VL_CONTA", "DS_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"}).drop("DS_CONTA", axis=1)

,1T2020
CNPJ_CIA,
00.001.180/0001-26,306836.0
00.070.698/0001-11,-2034.0
00.359.742/0001-08,3714.0
00.622.416/0001-41,34079.0
00.776.574/0001-56,-107963.0
...,...
92.781.335/0001-02,-157124.0
92.791.243/0001-03,17988.0
96.298.013/0001-68,11049.0
